In [2]:
from optimum.onnxruntime import ORTModelForSeq2SeqLM
from transformers import AutoTokenizer
from pathlib import Path

model_name = "digit82/kobart-summarization"
onnx_dir = Path("../../automation/models/kobart_summary_onnx")

# ✅ 1. 모델 변환 (export=True) ← 저장 안됨
model = ORTModelForSeq2SeqLM.from_pretrained(model_name, export=True)

# ✅ 2. ONNX 모델 저장
model.save_pretrained(onnx_dir)

# ✅ 3. 토크나이저 저장
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.save_pretrained(onnx_dir)

print("🎉 ONNX 변환 및 저장 완료:", onnx_dir)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.
c:\Users\user\anaconda3\envs\test-0602\lib\site-packages\transformers\models\bart\

🎉 ONNX 변환 및 저장 완료: ..\..\automation\models\kobart_summary_onnx


In [16]:
from optimum.onnxruntime import ORTModelForSeq2SeqLM
from tokenizers import Tokenizer
from pathlib import Path
import torch

def get_summarize_model():
    """
    ONNX 모델과 토크나이저를 반환하는 함수
    """

    model_dir = Path("../../automation/models/kobart_summary_onnx")
    model_summarize = ORTModelForSeq2SeqLM.from_pretrained(model_dir, local_files_only=True)

    tokenizer_summarize = Tokenizer.from_file(str(model_dir / "tokenizer.json"))

    return model_summarize, tokenizer_summarize

c:\Users\user\anaconda3\envs\test-0602\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import numpy as np
import onnxruntime as ort
from tokenizers import Tokenizer
from pathlib import Path

# ⬇ 모델 세션 초기화
def load_model():
    base_path = Path("../../automation/models/kobart_summary_onnx")
    encoder_sess = ort.InferenceSession(str(base_path / "encoder_model.onnx"))
    decoder_sess = ort.InferenceSession(str(base_path / "decoder_model.onnx"))
    tokenizer = Tokenizer.from_file(str(base_path / "tokenizer.json"))
    return encoder_sess, decoder_sess, tokenizer


# ⬇ 디코딩 함수 (greedy 또는 beam=1)
def summarize_event_focused(
    text,
    encoder_sess,
    decoder_sess,
    tokenizer,
    max_length=128,
    no_repeat_ngram_size=3,
    repetition_penalty=1.2,
):
    input_ids = tokenizer.encode(text).ids
    input_ids_np = np.array([input_ids], dtype=np.int64)
    attention_mask = np.ones_like(input_ids_np, dtype=np.int64)

    encoder_outputs = encoder_sess.run(
        None, {"input_ids": input_ids_np, "attention_mask": attention_mask}
    )[0]

    decoder_input_ids = [tokenizer.token_to_id("<s>")]
    generated_ids = decoder_input_ids.copy()

    for _ in range(max_length):
        decoder_input_np = np.array([generated_ids], dtype=np.int64)
        decoder_inputs = {
            "input_ids": decoder_input_np,
            "encoder_hidden_states": encoder_outputs,
            "encoder_attention_mask": attention_mask,
        }
        logits = decoder_sess.run(None, decoder_inputs)[0]
        next_token_logits = logits[:, -1, :]

        # repetition penalty 적용
        for token_id in set(generated_ids):
            next_token_logits[0, token_id] /= repetition_penalty

        # no_repeat_ngram_size 적용
        if no_repeat_ngram_size > 0 and len(generated_ids) >= no_repeat_ngram_size:
            ngram = tuple(generated_ids[-(no_repeat_ngram_size - 1) :])
            banned = {
                tuple(generated_ids[i : i + no_repeat_ngram_size])
                for i in range(len(generated_ids) - no_repeat_ngram_size + 1)
            }
            for token_id in range(next_token_logits.shape[-1]):
                if ngram + (token_id,) in banned:
                    next_token_logits[0, token_id] = -1e9  # 큰 마이너스

        # greedy 선택
        next_token_id = int(np.argmax(next_token_logits, axis=-1)[0])

        if next_token_id == tokenizer.token_to_id("</s>"):
            break

        generated_ids.append(next_token_id)

    return tokenizer.decode(generated_ids, skip_special_tokens=True)

In [37]:
encoder_sess, decoder_sess, tokenizer = load_model()

In [58]:
# 입력 문장
text = """
'사진=한경DB\n키움증권은 23일 삼성바이오로직스에 대해 인적분할을 추진하는 건 본격적인 신약 개발에 나서겠다는 의미하고 해석했다.\n삼성바이오로직스는 인적분할을 통해 의약품 위탁 개발·생산(CDMO) 부문은 존속법인에 남기고, 신설법인인 삼성에피스홀딩스에 삼성바이오에피스 지분 100%를 승계하도록 하는 방안을 추진하겠다고 공시했다.\n이에 대해 허혜민 키움증권 연구원은 “삼성그룹이 (삼성에피스홀딩스를 통해) 본격적으로 신약 개발을 시작한다는 의미로 해석된다”고 말했다. 이어 “삼성바이오로직스는 글로벌 파트너십 확대를 위한 영업활동이 수월해질 것”이라며 “분할된 신설법인은 투자지주회사로 신성장 동력 발굴과 연구·개발(R&D) 및 기업 인수·합병(M&A)를 통해 적극적인 성장을 추진할 것”이라고 내다봤다.\n이번 분할 추진이 단기적으로 펀더멘털에 미치는 영향은 미미할 것으로 분석됐다.\n다만 허 연구원은 “최근 인적분할 업체들의 분할상장 전 주가 수익률이 대체로 상승 흐름을 나타냈다”며 “이후 각 사의 펀더멘털, 수급, 모멘텀에 따라 변동했다”고 전했다.
"""

summary = summarize_event_focused(text, encoder_sess, decoder_sess, tokenizer)

print("📄 요약:", summary)

📄 요약: 삼성바이오로직스는 의약품 위탁 개발·생산(CDMO) 부문은 존속법인에 남기고, 신설법인인 삼성바이오에피스 지분 100%를 승계하도록 하는 방안을 추진하겠다고 밝히며 본격적인 신약 개발에 나서겠다는 의미하고 해석하였다.


In [29]:
import pandas as pd

# 행 전체 출력
pd.set_option("display.max_rows", None)

# 열 전체 출력
pd.set_option("display.max_columns", 200)

# 각 열 너비 무제한 출력
pd.set_option("display.max_colwidth", 200)

# 전체 넓이 제한 해제
pd.set_option("display.width", None)

In [18]:
import re
from kss import split_sentences

def remove_market_related_sentences(text: str) -> str:
    # 줄바꿈 제거
    text = text.replace("\n", " ")

    # 대괄호 포함 텍스트 제거: [파이낸셜뉴스], [사진] 등
    text = re.sub(r"\[[^\]]*\]", "", text)

    # '/사진', '/사진제공' 제거
    text = re.sub(r"/사진(제공)?", "", text)

    # 이메일 주소 제거 (예: josh@yna.co.kr)
    text = re.sub(r"\b[\w.-]+@[\w.-]+\.\w+\b", "", text)

    # 문장 단위 분리 (간단하게 마침표 기준, 필요시 KSS 등 적용 가능)
    sentences = split_sentences(text)

    # 제거할 패턴들 (뉴스 문장에서 자주 등장하는 패턴)
    patterns = [
        r"(자세한 내용|자세한 사항)",  # 뉴스 기본 표현
        r"\d{4}[.-]\d{1,2}[.-]\d{1,2}",  # 날짜 (예: 2025.03.26, 2024-12-01)
        r"([0-9,]+(?:만)?[0-9,]*\s?(?:원|만원))",  # 가격 (예: 3,500원, 12000원)
        r"(강세|펀드|시가총액|등락률|한국거래소)",  # 증시 용어
        r"\([+-]?[0-9.,]+%\)",  # 괄호 안 퍼센트 등락률
        r"(투자의견|연구원|평가|예상치|증권가|리포트|팀장)",  # 애널리스트 용어
        r"(순이익|전년|매출|영업이익|영업적자|증시|코스피|코스닥|다우|나스닥|매출액|거래일|호조세|레버리지|투자자|조정|자산|수익률|이익률|수익성|내리막|부진한|낙폭|기대치|실적발표|기업 가치)",  # 시장 용어
    ]

    # 하나의 통합 패턴으로 컴파일
    combined_pattern = re.compile("|".join(patterns))

    # 필터링된 문장만 유지
    filtered = [s for s in sentences if not combined_pattern.search(s)]

    text_preprocessed = " ".join(filtered)

    # print(f"원문:{sentences}\n|\n전처리 된 문장: {text_preprocessed}\n\n")

    return text_preprocessed

In [19]:
df_5 = df.head(5).copy()
df_5['article'] = df_5['article'].apply(remove_market_related_sentences)

[Kss]: Oh! You have mecab in your environment. Kss will take this as a backend! :D



In [39]:
df_5['article'][2]

NameError: name 'df_5' is not defined

In [38]:
from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForTokenClassification
from pathlib import Path

model_id = "KPF/KPF-BERT-NER"
save_dir = Path("../../automoation/models/ner_onnx")

# ONNX로 export
model = ORTModelForTokenClassification.from_pretrained(model_id, export=True)
model.save_pretrained(save_dir)

# tokenizer도 저장
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.save_pretrained(save_dir)

print("✅ NER ONNX 모델 저장 완료:", save_dir)

✅ NER ONNX 모델 저장 완료: ..\..\automoation\models\ner_onnx


In [ ]:
import onnxruntime as ort

def get_ner_tokenizer():
    """
    ONNX NER 모델을 위한 토크나이저를 반환합니다.
    """

    # 🟢 토크나이저 로딩
    tokenizer_ner = Tokenizer.from_file("../../automation/models/ner_onnx/tokenizer.json")

    # 🟢 ONNX 모델 세션 로딩
    session_ner = ort.InferenceSession("../../automation/models/ner_onnx/model.onnx")

    return tokenizer_ner, session_ner

In [67]:
import numpy as np
from label_map import id2label  # 라벨 ID ↔ 라벨명 매핑

tokenizer_ner, session_ner = get_ner_tokenizer()

def get_ner_tokens(tokenizer, session, text, id2label):
    # 🟡 토큰화 및 입력값 준비
    encoding = tokenizer.encode(text)
    input_ids = np.array([encoding.ids], dtype=np.int64)
    attention_mask = np.ones_like(input_ids, dtype=np.int64)
    token_type_ids = np.zeros_like(input_ids, dtype=np.int64)

    # 🔵 ONNX 추론 실행
    inputs = {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "token_type_ids": token_type_ids,
    }

    logits = session.run(None, inputs)[0]  # shape: (1, seq_len, num_labels)

    # 🔵 라벨 인덱스 → 실제 라벨명
    preds = np.argmax(logits, axis=-1)[0]
    labels = [id2label[p] for p in preds[: len(encoding.tokens)]]

    # 🔵 시각화
    tokens = encoding.tokens

    return tokens, labels

In [ ]:
def extract_ogg_economy(tokens, labels, target_label="OGG_ECONOMY"):
    merged_words = []
    current_word = ""

    for token, label in zip(tokens, labels):
        token_clean = token.replace("##", "") if token.startswith("##") else token

        if label == f"B-{target_label}":
            if current_word:
                merged_words.append(current_word)
            current_word = token_clean

        elif label == f"I-{target_label}":
            current_word += token_clean

        else:
            if current_word:
                merged_words.append(current_word)
                current_word = ""

    if current_word:
        merged_words.append(current_word)

    stock_list = merged_words.copy()

    return stock_list

In [71]:
import numpy as np
from label_map import id2label  # 라벨 ID ↔ 라벨명 매핑

text = """
세계 2위 투자은행(IB)인 골드만삭스의 사장 겸 최고운영책임자(COO)인 존 월드론이 방한해 이창용 한국은행 총재, 이재용 삼성전자 회장, 김병주 MBK파트너스 회장 등과 함께 한국의 금융당국, 기업 및 금융계와의 협력 방안을 논의했다.
"""

tokens, labels = get_ner_tokens(tokenizer_ner, session_ner, text, id2label)
stock_list = extract_ogg_economy(tokens, labels, target_label="OGG_ECONOMY")
stock_list

['골드만삭스', '한국은행', '삼성전자', 'MBK파트너스']

In [57]:
import pandas as pd

df = pd.read_csv('../../db/news_2023_2025_summarized_view.csv')
df['article'][20]

'사진=한경DB\n키움증권은 23일 삼성바이오로직스에 대해 인적분할을 추진하는 건 본격적인 신약 개발에 나서겠다는 의미하고 해석했다.\n삼성바이오로직스는 인적분할을 통해 의약품 위탁 개발·생산(CDMO) 부문은 존속법인에 남기고, 신설법인인 삼성에피스홀딩스에 삼성바이오에피스 지분 100%를 승계하도록 하는 방안을 추진하겠다고 공시했다.\n이에 대해 허혜민 키움증권 연구원은 “삼성그룹이 (삼성에피스홀딩스를 통해) 본격적으로 신약 개발을 시작한다는 의미로 해석된다”고 말했다. 이어 “삼성바이오로직스는 글로벌 파트너십 확대를 위한 영업활동이 수월해질 것”이라며 “분할된 신설법인은 투자지주회사로 신성장 동력 발굴과 연구·개발(R&D) 및 기업 인수·합병(M&A)를 통해 적극적인 성장을 추진할 것”이라고 내다봤다.\n이번 분할 추진이 단기적으로 펀더멘털에 미치는 영향은 미미할 것으로 분석됐다.\n다만 허 연구원은 “최근 인적분할 업체들의 분할상장 전 주가 수익률이 대체로 상승 흐름을 나타냈다”며 “이후 각 사의 펀더멘털, 수급, 모멘텀에 따라 변동했다”고 전했다.'

In [13]:
df = df[['news_id', 'summary', 'stock_list', 'industry_list']]
df.head()

,news_id,summary,stock_list,industry_list
0,20250523_0002,23일 정보기술(IT)·투자은행(IB) 업계에 따르면 국내 대표 전자결제사업자인 카...,['카카오페이'],['금융 지원 서비스업']
1,20250523_0004,세계 2위 투자은행(IB)인 골드만삭스의 사장 겸 최고운영책임자(COO)인 존 월드...,['삼성전자'],['통신 및 방송 장비 제조업']
2,20250523_0007,23일 정보기술(IT)·투자은행(IB) 업계에 따르면 카카오페이가 SSG닷컴 쓱페이...,['카카오페이'],['금융 지원 서비스업']
3,20250523_0010,23일 효성중공업은 조 회장이 상속세 재원을 마련하기 위해 시간 외 매매로 효성중 ...,['효성중공업'],"['전동기, 발전기 및 전기 변환 · 공급 · 제어 장치 제조업']"
4,20250523_0011,카카오페이가 신세계 이마트 측에서 쓱페이·스마일페이 인수를 추진하고 나선 건 국내 ...,['카카오페이'],['금융 지원 서비스업']


In [14]:
df.to_csv('../../db/news_2023_2025_metadata.csv', index=False)